# Steganography

## Module Import

In [2]:
import wave
import numpy as np
import os
from pydub import AudioSegment

## Audio Conversion Function

Now we define a function that check the imported cover audio if it has '.wav' extension. If not, we convert it using PyDub.

In [3]:
def convertToWav(inputFile):
    if not os.path.exists(inputFile):
        print(f"Error: The input file '{inputFile}' does not exist.")
        return None
    _, ext = os.path.splitext(inputFile)
    if ext.lower() == '.wav':
        print(f"The file '{inputFile}' is already in WAV format.")
        return inputFile
    outputFile = os.path.splitext(inputFile)[0] + '.wav'
    try:
        audio = AudioSegment.from_file(inputFile)
        audio.export(outputFile, format="wav")
        print(f"Audio successfully converted to WAV format: {outputFile}")
        return outputFile
    except Exception as e:
        print(f"An error occurred during conversion: {e}")
        return None

We do the same function on the message audio.

## Capacity Checking Function

As cover audio can hold only so much of message audio, it is crucial to check for overflow prior performing steganography.

In [4]:
def calculate_audio_capacity(file_path):
    """Calculate the bit capacity of the cover audio."""
    with wave.open(file_path, 'rb') as wf:
        # Extract properties
        n_channels = wf.getnchannels()          # Number of channels
        sample_width = wf.getsampwidth()       # Bytes per sample
        frame_rate = wf.getframerate()         # Sampling rate (samples per second)
        n_frames = wf.getnframes()             # Total frames
        bit_depth = sample_width * 8           # Bits per sample
        duration = n_frames / frame_rate       # Duration in seconds

        # Calculate total number of samples
        total_samples = n_frames * n_channels  # Each channel counts as a sample

        # Capacity is 1 bit per sample for LSB steganography
        capacity_bits = total_samples          # Total bits available for LSB

        print(f"Cover Audio Properties:")
        print(f"  Duration: {duration:.2f} seconds")
        print(f"  Channels: {n_channels}")
        print(f"  Bit Depth: {bit_depth} bits")
        print(f"  Frame Rate: {frame_rate} Hz")
        print(f"  Total Samples: {total_samples}")
        print(f"  Capacity for LSB Steganography: {capacity_bits} bits")

        return capacity_bits, duration


def calculate_message_size(file_path):
    """Calculate the bit size of the message audio."""
    with wave.open(file_path, 'rb') as wf:
        # Extract properties
        n_channels = wf.getnchannels()
        sample_width = wf.getsampwidth()
        frame_rate = wf.getframerate()
        n_frames = wf.getnframes()
        bit_depth = sample_width * 8           # Bits per sample
        duration = n_frames / frame_rate       # Duration in seconds

        # Total number of samples in the message audio
        total_samples = n_frames * n_channels

        # Total bits in the message
        total_bits = total_samples * bit_depth

        print(f"Message Audio Properties:")
        print(f"  Duration: {duration:.2f} seconds")
        print(f"  Channels: {n_channels}")
        print(f"  Bit Depth: {bit_depth} bits")
        print(f"  Frame Rate: {frame_rate} Hz")
        print(f"  Total Samples: {total_samples}")
        print(f"  Message Size: {total_bits} bits")

        return total_bits


def check_capacity(cover_audio_path, message_audio_path):
    """Check if the message audio can fit into the cover audio."""
    if not os.path.exists(cover_audio_path):
        print(f"Error: Cover audio file '{cover_audio_path}' not found.")
        return False
    if not os.path.exists(message_audio_path):
        print(f"Error: Message audio file '{message_audio_path}' not found.")
        return False

    # Calculate cover audio capacity
    cover_capacity, cover_duration = calculate_audio_capacity(cover_audio_path)

    # Calculate message audio size
    message_size = calculate_message_size(message_audio_path)

    # Compare capacities
    if cover_capacity >= message_size:
        print(f"\nSuccess: The message audio can fit into the cover audio!")
        print(f"  Cover Audio Capacity: {cover_capacity} bits")
        print(f"  Message Audio Size: {message_size} bits")
        return True
    else:
        print(f"\nError: The message audio is too large to fit into the cover audio.")
        print(f"  Cover Audio Capacity: {cover_capacity} bits")
        print(f"  Message Audio Size: {message_size} bits")
        print(f"  You would need a larger cover audio or a smaller message audio.")
        return False

## Audio to Binary Conversion Function

Now, we define a function that will convert both the cover and message audio as per their metadata.

In [5]:
def audio_to_binary(audio_path):
    """Convert audio samples to a binary stream."""
    with wave.open(audio_path, 'rb') as wf:
        n_channels = wf.getnchannels()
        sample_width = wf.getsampwidth()
        n_frames = wf.getnframes()

        # Read raw frames and convert to integers
        raw_frames = wf.readframes(n_frames)
        # Maximum integer based on sample width
        max_int = 2 ** (sample_width * 8)

        # Convert raw bytes to integers, taking sample width into account
        samples = [
            int.from_bytes(raw_frames[i:i+sample_width],
                           'little', signed=(sample_width > 1))
            for i in range(0, len(raw_frames), sample_width)
        ]

        # Convert each sample to binary
        binary_stream = ''.join(
            f'{(sample + max_int) % max_int:0{sample_width * 8}b}' for sample in samples)
        return binary_stream, n_channels

## Channel-wise LSB Steganography Function

Now, we define a function that will perform channel-wise lsb steganography, that is, encode the message audio in the cover audio.

In [6]:
def encode_lsb(cover_audio_path, message_audio_path, output_path):
    """Perform LSB steganography to encode the message audio into the cover audio."""
    # Load the cover audio and message audio
    with wave.open(cover_audio_path, 'rb') as cover_wav, wave.open(message_audio_path, 'rb') as message_wav:
        # Cover audio properties
        cover_channels = cover_wav.getnchannels()
        cover_sample_width = cover_wav.getsampwidth()
        cover_frame_rate = cover_wav.getframerate()
        cover_frames = cover_wav.getnframes()

        # Message audio binary stream
        message_binary, message_channels = audio_to_binary(message_audio_path)

        # Ensure the message can fit into the cover
        total_cover_samples = cover_frames * cover_channels
        if len(message_binary) > total_cover_samples:
            raise ValueError(
                "Message audio is too large to fit into the cover audio.")

        # Read cover audio frames
        raw_cover_frames = cover_wav.readframes(cover_frames)
        cover_samples = [
            int.from_bytes(
                raw_cover_frames[i:i+cover_sample_width], 'little', signed=(cover_sample_width > 1))
            for i in range(0, len(raw_cover_frames), cover_sample_width)
        ]

        # Encode the message into the LSB of cover audio
        message_index = 0
        # Maximum integer based on sample width
        max_int = 2 ** (cover_sample_width * 8)
        for i in range(len(cover_samples)):
            if message_index < len(message_binary):
                # Modify the LSB of the cover sample
                cover_samples[i] = (cover_samples[i] & ~1) | int(
                    message_binary[message_index])
                message_index += 1

        # Convert back to bytes
        stego_frames = b''.join(
            ((sample + max_int) % max_int).to_bytes(cover_sample_width,
                                                    'little', signed=(cover_sample_width > 1))
            for sample in cover_samples
        )

        # Write the stego audio to a new WAV file
        with wave.open(output_path, 'wb') as stego_wav:
            stego_wav.setnchannels(cover_channels)
            stego_wav.setsampwidth(cover_sample_width)
            stego_wav.setframerate(cover_frame_rate)
            stego_wav.writeframes(stego_frames)

    print(f"Message audio has been encoded into the cover audio and saved to {
          output_path}")

Now, we define a function that will decrypt the stego audio and retrieve the message audio. However, in decryption we have to specify the bit depth and channels in the message audio, as the code will not work otherwise. It is assumed that a mono audio signal with 16 bit depth is the message signal.

In [7]:
def decode_lsb(stego_audio_path, message_audio_path, message_bit_depth=16, message_channels=1):
    """
    Decode the hidden message audio from the stego audio using LSB decoding.

    Args:
        stego_audio_path (str): Path to the stego audio file.
        message_audio_path (str): Path where the decoded message audio will be saved.
        message_bit_depth (int): Bit depth of the hidden message audio (e.g., 16).
        message_channels (int): Number of channels in the hidden message audio (e.g., 1 for mono).

    Returns:
        None
    """
    with wave.open(stego_audio_path, 'rb') as stego_wav:
        # Extract stego audio properties
        stego_channels = stego_wav.getnchannels()
        stego_sample_width = stego_wav.getsampwidth()
        stego_frame_rate = stego_wav.getframerate()
        stego_frames = stego_wav.getnframes()

        # Read all samples from the stego audio
        raw_stego_frames = stego_wav.readframes(stego_frames)
        # Maximum integer based on sample width
        max_int = 2 ** (stego_sample_width * 8)

        # Extract samples as integers
        stego_samples = [
            int.from_bytes(
                raw_stego_frames[i:i+stego_sample_width], 'little', signed=(stego_sample_width > 1))
            for i in range(0, len(raw_stego_frames), stego_sample_width)
        ]

        # Extract LSBs from stego samples
        binary_stream = ''.join(str(sample & 1) for sample in stego_samples)

        # Reconstruct message samples from binary stream
        message_samples = [
            int(binary_stream[i:i+message_bit_depth], 2) -
            (2 ** (message_bit_depth - 1))
            for i in range(0, len(binary_stream), message_bit_depth)
            if i + message_bit_depth <= len(binary_stream)
        ]

        # Adjust for multi-channel message audio
        message_frames = len(message_samples) // message_channels
        message_samples = message_samples[:message_frames * message_channels]

        # Write the reconstructed message audio to a new file
        with wave.open(message_audio_path, 'wb') as message_wav:
            message_sample_width = message_bit_depth // 8  # Convert bit depth to bytes
            message_wav.setnchannels(message_channels)
            message_wav.setsampwidth(message_sample_width)
            message_wav.setframerate(stego_frame_rate)
            message_wav.writeframes(
                b''.join(
                    (sample + max_int // 2).to_bytes(message_sample_width,
                                                     'little', signed=(message_sample_width > 1))
                    for sample in message_samples
                )
            )

    print(f"Hidden message audio has been decoded and saved to {
          message_audio_path}")